<a href="https://colab.research.google.com/github/MartonJToth/opengl_pytorch/blob/master/opengl_pytorch_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install libglfw3
!apt-get install libglfw3-dev
!apt-get install xvfb

!pip install xvfbwrapper
!pip install PyVirtualDisplay
!pip install PyOpenGL
!pip install glfw

Hasznos importok

In [0]:
import numpy as np
from OpenGL.GL import *
from OpenGL.GLU import *
import glfw
from PIL import Image
import PIL
import matplotlib.pyplot as plt

from pyvirtualdisplay import Display as VirtDisplay

import torch
from torch.utils.data import Dataset
import torch.nn as nn
from torch.utils.data import DataLoader

from skimage.color import rgb2lab, lab2rgb

from collections import namedtuple
import torchvision.models.vgg as vgg

Néhány konstans megadás

In [0]:
DISPLAY_WIDTH = 300
DISPLAY_HEIGHT = 300

light_ambient = [0.1, 0.1, 0.1, 0.1]
light_diffuse = [0.5, 0.5, 0.5, 1]
light_position = [5, 5, 5, 0]

torch.manual_seed(999)
torch.cuda.manual_seed_all(999)

OpenGL inicializálás és erőforrás felszabadítás

In [0]:
def init():
  
  display = VirtDisplay(visible=0, size=(1280, 1024))
  display.start()

  if not glfw.init():
    return

  glfw.window_hint(glfw.VISIBLE, False)
  window = glfw.create_window(DISPLAY_WIDTH, DISPLAY_HEIGHT, 'hidden_window', None, None)

  if not window:
    glfw.terminate()
    return

  glfw.make_context_current(window)

  glViewport(0, 0, DISPLAY_WIDTH, DISPLAY_HEIGHT)
  glMatrixMode( GL_PROJECTION )
  glLoadIdentity()
  gluPerspective(90, (DISPLAY_WIDTH / DISPLAY_HEIGHT), 0.01, 100)

  glEnable(GL_TEXTURE_2D)
  glEnable(GL_DEPTH_TEST)
  glDepthFunc(GL_LEQUAL)

  glLightfv(GL_LIGHT0, GL_AMBIENT, light_ambient)
  glLightfv(GL_LIGHT0, GL_POSITION, light_position)
  glLightfv(GL_LIGHT0, GL_DIFFUSE, light_diffuse)
  glEnable(GL_LIGHT0)
  glEnable(GL_LIGHTING)
  
  glMatrixMode( GL_MODELVIEW )
  glLoadIdentity()

  glClearColor( 0.0, 0.0, 0.0, 1.0 )


In [0]:
def dispose(window):
  
  glfw.destroy_window(window)
  glfw.terminate()
  
  return

In [0]:
window = init()

Rajzoló függvény

In [0]:
def renderCube(x, y, z, r, g, b):

  glPushMatrix()
  
  glTranslatef( x, y, z );
  
  glBegin(GL_QUADS)
  
  glColor3f( r, g, b )
  
  glNormal3f(0, 0, -1)
  glVertex3f( -0.5, -0.5 , -0.5)
  glVertex3f(  0.5, -0.5 , -0.5)
  glVertex3f(  0.5,  0.5 , -0.5)
  glVertex3f( -0.5,  0.5 , -0.5)
  
  glNormal3f(0, 0, 1)
  glVertex3f( -0.5,  0.5 , 0.5)
  glVertex3f(  0.5,  0.5 , 0.5)
  glVertex3f(  0.5, -0.5 , 0.5)
  glVertex3f( -0.5, -0.5 , 0.5)
  
  glNormal3f(0, -1, 0)
  glVertex3f( -0.5, -0.5 , -0.5)
  glVertex3f(  0.5, -0.5 , -0.5)
  glVertex3f(  0.5, -0.5 ,  0.5)
  glVertex3f( -0.5, -0.5 ,  0.5)
  
  glNormal3f(0, 1, 0)
  glVertex3f( -0.5, 0.5 , -0.5)
  glVertex3f(  0.5, 0.5 , -0.5)
  glVertex3f(  0.5, 0.5 ,  0.5)
  glVertex3f( -0.5, 0.5 ,  0.5)
  
  glNormal3f(-1, 0, 0)
  glVertex3f( -0.5, -0.5 ,  -0.5)
  glVertex3f( -0.5,  0.5 ,  -0.5)
  glVertex3f( -0.5,  0.5 ,   0.5)
  glVertex3f( -0.5, -0.5 ,   0.5)
  
  glNormal3f(1, 0, 0)
  glVertex3f( 0.5, -0.5 ,  -0.5)
  glVertex3f( 0.5,  0.5 ,  -0.5)
  glVertex3f( 0.5,  0.5 ,   0.5)
  glVertex3f( 0.5, -0.5 ,   0.5)
  
  glEnd()
  
  glPopMatrix()

def render(rx, widht, height):
  
  glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
  
  glPushMatrix()
  
  gluLookAt(0, 0, 1.5,
            0, 0, -1,
            0, 1, 0)
  
  glTranslatef( 0.0, 0.0, 0.0 )
  
  glRotatef(rx, 1, 0, 0)
  
  glColorMaterial(GL_FRONT_AND_BACK, GL_AMBIENT_AND_DIFFUSE)
  glEnable(GL_COLOR_MATERIAL)
  
  #renderCube(0, 0, 0, 1, 0, 0)
  #renderCube(0, 1, 0, 0, 1, 0)
  #renderCube(0, -1, 0, 0, 0, 1)

  glBegin(GL_QUADS)  
  
  glColor3f( 1.0, 0.0, 0.0 )
  glNormal3f(0, 0, -1)
  glVertex3f( -0.5, -0.5 , -0.5)
  glVertex3f(  0.5, -0.5 , -0.5)
  glVertex3f(  0.5,  0.5 , -0.5)
  glVertex3f( -0.5,  0.5 , -0.5)
  
  glColor3f( 0.0, 1.0, 0.0 )
  glNormal3f(0, 0, 1)
  glVertex3f( -0.5,  0.5 , 0.5)
  glVertex3f(  0.5,  0.5 , 0.5)
  glVertex3f(  0.5, -0.5 , 0.5)
  glVertex3f( -0.5, -0.5 , 0.5)
  
  glColor3f( 1.0, 0.0, 0.0 )
  glNormal3f(0, -1, 0)
  glVertex3f( -0.5, -0.5 , -0.5)
  glColor3f( 0.0, 1.0, 0.0 )
  glVertex3f(  0.5, -0.5 , -0.5)
  glColor3f( 0.0, 0.0, 1.0 )
  glVertex3f(  0.5, -0.5 , 0.5)
  glColor3f( 1.0, 1.0, 1.0 )
  glVertex3f( -0.5, -0.5 , 0.5)
  
  
  glColor3f( 0.0, 1.0, 0.0 )
  glNormal3f(0, 1, 0)
  glVertex3f( -0.5, 0.5 , -0.5)
  glColor3f( 0.0, 1.0, 0.0 )
  glVertex3f(  0.5, 0.5 , -0.5)
  glColor3f( 1.0, 1.0, 0.0 )
  glVertex3f( 0.5, 0.5 , 0.5)
  glColor3f( 1.0, 1.0, 0.0 )
  glVertex3f(  -0.5, 0.5 , 0.5)
  
  glEnd()
  
  glPopMatrix()
  
  image_buffer = glReadPixels(0, 0, widht, height, OpenGL.GL.GL_RGB, OpenGL.GL.GL_UNSIGNED_BYTE)
  image = np.frombuffer(image_buffer, dtype=np.uint8).reshape(widht, height, 3)

  return image

In [0]:
image = render(50, DISPLAY_WIDTH, DISPLAY_HEIGHT)

plt.figure(1)
plt.imshow(image)
plt.show()

Adat betöltő osztály

In [0]:
class ToTensor(object):

    def __call__(self, sample):
        input, output = sample['input'], sample['output']
        
        inp = np.zeros(1)
        inp[0] = input
        
        output = np.divide(output, 255.0)

        return {'input': torch.from_numpy(inp).float(),
                'output': torch.from_numpy(output).float()}

class RenderDataset(Dataset):
  
  def __init__(self, width, height, transform=None, training=True):
    super(RenderDataset, self).__init__()
    
    self.transform = transform
    self.width = width
    self.height = height
    self.training = training
    self.divider = 10
    
  def __len__(self):
    return 360
  
  def __getitem__(self, idx):
    
    #TODO: Renderelés meghívása
    
    
    y = np.zeros((3, DISPLAY_WIDTH, DISPLAY_HEIGHT))
    y = np.transpose(y, (2, 1, 0))
    
    sample = {'input': idx, 'output': y}

    if self.transform:
        sample = self.transform(sample)

    return sample

Egyszerű hálózat

In [0]:
class Decoder(nn.Module):
  def __init__(self, latent_size, output_size):
    super(Decoder, self).__init__()

    self.latent_size = latent_size
    self.output_size = output_size

    self.fc = nn.Linear(self.latent_size, 256)

    self.cnn_decoder = nn.Sequential(
        nn.ConvTranspose2d(1, 32, kernel_size=4, stride=2, padding=2, dilation=1),
        nn.ELU(),

        nn.ConvTranspose2d(32, 64, kernel_size=4, stride=2, padding=2, dilation=1),
        nn.ELU(),

        nn.ConvTranspose2d(64, 64, kernel_size=4, stride=2, padding=2, dilation=1),
        nn.ELU(),
        
        nn.ConvTranspose2d(64, 64, kernel_size=4, stride=2, padding=2, dilation=1),
        nn.ELU(),
        
        nn.ConvTranspose2d(64, 3, kernel_size=5, stride=2, padding=1, dilation=1),
        nn.ELU(),
    )
        
  def forward(self, latent_input):
    #----------------------------------------------------------------
    out = self.fc(latent_input)
    out = out.view(out.size(0), 1, 16, 16)
    out = self.cnn_decoder(out)
    out = out[0:out.size(0), 0:3, 0:self.output_size[0], 0:self.output_size[1]]
    #----------------------------------------------------------------
    
    return out

Egy lehetsége saját loss függvény

In [0]:
class MyLoss(nn.Module):

    def __init__(self, cuda):
        super(MyLoss, self).__init__()

        a = np.array([[1, 0, -1], [2, 0, -2], [1, 0, -1]])
        self.conv1 = nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1, bias=False)
        self.conv1.weight = nn.Parameter(torch.from_numpy(a).float().unsqueeze(0).unsqueeze(0))

        b = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])
        self.conv2 = nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1, bias=False)
        self.conv2.weight = nn.Parameter(torch.from_numpy(b).float().unsqueeze(0).unsqueeze(0))

        self.cuda = cuda
        if cuda:
            self.conv1.cuda()
            self.conv2.cuda()

    def getLuminance(self, x):

        L = torch.autograd.Variable(torch.zeros([x.size()[0], 1, x.size()[2], x.size()[3]]), requires_grad=False)

        if self.cuda:
            L = L.cuda()

        L[:, 0, :, :] = 0.2126 * x[:, 0, :, :] + 0.7152 * x[:, 1, :, :] + 0.0722 * x[:, 2, :, :]

        return L

    def forward(self, x, y):

        Lx = self.getLuminance(x)
        Ly = self.getLuminance(y)

        dLxx = self.conv1(Lx)
        dLxy = self.conv2(Lx)

        dLyx = self.conv1(Ly)
        dLyy = self.conv2(Ly)

        diff = (dLxx - dLyx) * (dLxx - dLyx) + (dLxy - dLyy) * (dLxy - dLyy) + (x - y) * (x - y)
        diff = torch.sum(diff) / x.numel()

        loss = diff 
        
        return loss
      
myLoss = MyLoss(True)

VGG16 feature alapú loss

In [0]:
LossOutput = namedtuple("LossOutput", ["relu1_2", "relu2_2", "relu3_3", "relu4_3"])
# https://discuss.pytorch.org/t/how-to-extract-features-of-an-image-from-a-trained-model/119/3
class LossNetwork(torch.nn.Module):
    def __init__(self, vgg_model):
        super(LossNetwork, self).__init__()
        self.vgg_layers = vgg_model.features
        self.layer_name_mapping = {
            '3': "relu1_2",
            '8': "relu2_2",
            '15': "relu3_3",
            '22': "relu4_3"
        }
    
    def forward(self, x):
        output = {}
        for name, module in self.vgg_layers._modules.items():
            x = module(x)
            if name in self.layer_name_mapping:
                output[self.layer_name_mapping[name]] = x
        return LossOutput(**output)
      
      
vgg_model = vgg.vgg16(pretrained=True)
vgg_model.cuda()
loss_network = LossNetwork(vgg_model)
loss_network.cuda()
loss_network.eval()

In [0]:
Tanító metódus

In [0]:
def training():
  
  for epoch in range(100):
  
    running_loss = 0.0
    net.train()

    for i, data in enumerate(trainLoader, 0):

      input, output = data['input'], data['output']
      input, output = input.cuda(), output.cuda()

      optimizer.zero_grad()

      result = net(input)
      loss = mse(result, output)

      loss.backward()
      optimizer.step()

      running_loss += loss.item()

    tr_loss = running_loss / len(trainLoader)
    print("Train epoch %d loss: %.3f" % (epoch + 1, tr_loss))
    

In [0]:
tt = ToTensor()
trainingDS = RenderDataset(width = DISPLAY_WIDTH, height = DISPLAY_HEIGHT, transform = tt)
trainLoader = DataLoader(trainingDS, batch_size=16, shuffle=True, num_workers=0)

net = Decoder(1, (DISPLAY_WIDTH, DISPLAY_HEIGHT))
net.cuda()

mse = torch.nn.MSELoss()

In [0]:
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

training()

Vizuális kiértékelés

In [0]:
evalDS = RenderDataset(width = DISPLAY_WIDTH, height = DISPLAY_HEIGHT, transform = tt, training=False)

idx = 135

net.eval()

image = evalDS[idx]
input = np.zeros((1, 1))
input[0, 0] = image['input']
result = net(torch.from_numpy(input).float().cuda())
npResult = result.detach().cpu().numpy()[0, :, :, :];

plt.figure(1)
plt.subplot(2, 2, 1)
plt.imshow(np.transpose(npResult, (2, 1, 0)))
plt.subplot(2, 2, 2)
plt.imshow(np.transpose(image['output'].numpy(), (2, 1, 0)))

image = evalDS[idx + 9]
input = np.zeros((1, 1))
input[0, 0] = image['input']
result = net(torch.from_numpy(input).float().cuda())
npResult = result.detach().cpu().numpy()[0, :, :, :];
print(npResult.shape)

plt.figure(1)
plt.subplot(2, 2, 3)
plt.imshow(np.transpose(npResult, (2, 1, 0)))
plt.subplot(2, 2, 4)
plt.imshow(np.transpose(image['output'].numpy(), (2, 1, 0)))

plt.show()